In [1]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from collections import deque 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import plotly.express as px

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [6]:
X,y = make_classification(15_000,500,random_state=50,weights=[0.9])
X, X_validate, y, y_validate = train_test_split(X,y, random_state=50)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =  0.7)

In [8]:
# X_train_ = X_train.copy()
# y_train_ = y_train.copy()
# rng_idx = pd.Series(y_train).sample(len(y_train)).index
# X_train_ = X_train_[rng_idx]
# y_train_ = y_train_[rng_idx]
# M_idx, m_idx = pd.DataFrame(y_train_).groupby(0).apply(lambda x: x.index.values)

In [9]:
# pd.Series(y_train).value_counts()   

In [10]:
# bal_idx = pd.DataFrame(y_train, columns = ['y']).groupby('y').apply(lambda x: x.sample(250)).index.to_frame()[1].values


In [11]:
# rng = np.random.default_rng(1235)

In [12]:
# rng.shuffle(bal_idx)

In [13]:
# X_bal = X_train[bal_idx]
# y_bal = y_train[bal_idx]

In [235]:
# X_bal = X_train.copy()
# y_bal = y_train.copy()

In [236]:
# M_idx, m_idx = pd.DataFrame(y_bal).groupby(0).apply(lambda x: x.index.values)

---

In [262]:
%run -i ./DQN_Class.py
gamma=0.5
epsilon=.8
numberEpisodes= 120
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


# NN approach

In [263]:

nn = LearningQDeep.buildNetwork()

In [264]:
nn.summary()

Model: "sequential_95"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_355 (Dense)           (None, 30)                15030     
                                                                 
 dense_356 (Dense)           (None, 64)                1984      
                                                                 
 dropout_19 (Dropout)        (None, 64)                0         
                                                                 
 dense_357 (Dense)           (None, 20)                1300      
                                                                 
 dense_358 (Dense)           (None, 1)                 21        
                                                                 
Total params: 18,335
Trainable params: 18,335
Non-trainable params: 0
_________________________________________________________________


In [265]:

early_stopping = tf.keras.callbacks.EarlyStopping(
                                monitor='val_f1', 
                                verbose=1,
                                patience=100,
                                mode='max',
                                restore_best_weights=True)

In [266]:
nn.fit(X_train,y_train, validation_data=[X_test,y_test], epochs=500, callbacks = [early_stopping])

Epoch 1/500
106/106 [==============================] - 2s 7ms/step - loss: 0.4291 - recall_m: 0.1144 - precision_m: 0.0462 - f1: 0.0513 - val_loss: 0.3416 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 2/500
106/106 [==============================] - 0s 4ms/step - loss: 0.3112 - recall_m: 0.0267 - precision_m: 0.0943 - f1: 0.0398 - val_loss: 0.3233 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1: 0.0000e+00
Epoch 3/500
106/106 [==============================] - 0s 5ms/step - loss: 0.2560 - recall_m: 0.0598 - precision_m: 0.1462 - f1: 0.0794 - val_loss: 0.3043 - val_recall_m: 0.0095 - val_precision_m: 0.0283 - val_f1: 0.0128
Epoch 4/500
106/106 [==============================] - 0s 4ms/step - loss: 0.2061 - recall_m: 0.2113 - precision_m: 0.4371 - f1: 0.2641 - val_loss: 0.2875 - val_recall_m: 0.1013 - val_precision_m: 0.2618 - val_f1: 0.1369
Epoch 5/500
106/106 [==============================] - 0s 5ms/step - loss: 0.1515 - recall_m

In [272]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_train)]

106/106 [==============================] - 0s 2ms/step


In [273]:
print(confusion_matrix(y_train, y_pred))

[[3056    0]
 [   0  319]]


In [274]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3056
           1       1.00      1.00      1.00       319

    accuracy                           1.00      3375
   macro avg       1.00      1.00      1.00      3375
weighted avg       1.00      1.00      1.00      3375



In [275]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_test)]

247/247 [==============================] - 0s 2ms/step


In [276]:
print(confusion_matrix(y_test, y_pred))

[[6795  225]
 [ 438  417]]


In [277]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7020
           1       0.65      0.49      0.56       855

    accuracy                           0.92      7875
   macro avg       0.79      0.73      0.76      7875
weighted avg       0.91      0.92      0.91      7875



In [278]:
y_pred = [1 if x>0.5 else 0 for x in nn.predict(X_validate)]

118/118 [==============================] - 0s 1ms/step


In [279]:
print(confusion_matrix(y_validate, y_pred))

[[3255  118]
 [ 191  186]]


In [280]:
print(classification_report(y_validate, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      3373
           1       0.61      0.49      0.55       377

    accuracy                           0.92      3750
   macro avg       0.78      0.73      0.75      3750
weighted avg       0.91      0.92      0.91      3750



---

In [146]:
 actions = {0 : 0, 1 : 1}
 len(actions)

2

In [147]:
def reward(si, a, M, m, y):
    l = y[si]

    if (a == l) & (si in m):
        r = 1
    elif  (a != l) & (si in m):
        r = -1
    if (a == l) & (si in M):
        r = 0.1
    elif  (a != l) & (si in M):
        r = -0.1
    return r

In [148]:
def step(action, i, M, m, X, y):

    r = reward(i,action, M, m, y)

    nextState = X[i+1,:]

    terminated = False
    
    l = y[i]

    if i == len(y) - 1:
        terminated = True
    elif (action != l) & (i in m):
        print(action, l)
        terminated = True


    return r, nextState, terminated, i+1
    

In [149]:
%run -i ./DQN_Class.py
gamma=0.2
epsilon=.9
numberEpisodes= 500
LearningQDeep=DeepQLearning(gamma,epsilon,numberEpisodes)


In [150]:
0.96**(500-25)*(0.9)

3.4125435562551007e-09

In [151]:
LearningQDeep.actionDimension

2

In [152]:
LearningQDeep.epsilon

0.9

In [153]:
LearningQDeep.mainNetwork.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_124 (Dense)           (None, 30)                15030     
                                                                 
 dense_125 (Dense)           (None, 64)                1984      
                                                                 
 dense_126 (Dense)           (None, 20)                1300      
                                                                 
 dense_127 (Dense)           (None, 2)                 42        
                                                                 
Total params: 18,356
Trainable params: 18,356
Non-trainable params: 0
_________________________________________________________________


In [154]:

%%time
LearningQDeep.trainingEpisodes()

Simulating episode 0
	 First train of main network...
0 1
	 First 1 index: [ 18  25  47  51  61  68  69  70 114 127]
	 Reached sample: 19
	 Sum of rewards -1.2
Simulating episode 1
	 First train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [23 29 30 38 45 50 60 67 78 88]
	 Reached sample: 24
	 Sum of rewards 0.29999999999999993
Simulating episode 2
	 First train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 20  78  93 130 151 154 163 171 173 176]
	 Reached sample: 21
	 Sum of rewards -0.6
Simulating episode 3
	 First train of main network...
0 1
	 First 1 index: [13 27 32 39 43 44 73 75 85 87]
	 Reached sample: 14
	 Sum of rewards -0.4999999999999999
Simulating episode 4
	 First train of main network...
		 [UPDATED] Target network
		 Current epsilon: 0.9
0 1
	 First 1 index: [ 9 15 21 23 25 29 36 38 41 57]
	 Reached sample: 16
	 Sum of rewar

In [155]:
loaded_model = LearningQDeep.mainNetwork

In [156]:
y_pred  = np.argmax(loaded_model.predict(X_train,verbose=0), axis = 1)

In [157]:

print(confusion_matrix(y_train, y_pred))

[[2957   76]
 [  33  309]]


In [158]:
print(classification_report(y_train,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      3033
           1       0.80      0.90      0.85       342

    accuracy                           0.97      3375
   macro avg       0.90      0.94      0.92      3375
weighted avg       0.97      0.97      0.97      3375



In [159]:
y_pred  = np.argmax(loaded_model.predict(X_test,verbose=0), axis = 1)

In [160]:
print(confusion_matrix(y_test, y_pred))

[[6361  682]
 [ 272  560]]


In [161]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      7043
           1       0.45      0.67      0.54       832

    accuracy                           0.88      7875
   macro avg       0.70      0.79      0.74      7875
weighted avg       0.91      0.88      0.89      7875



In [162]:
y_pred  = np.argmax(loaded_model.predict(X_validate,verbose=0), axis = 1)

In [163]:
print(confusion_matrix(y_validate, y_pred))

[[3009  364]
 [ 112  265]]


In [164]:
print(classification_report(y_validate,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.89      0.93      3373
           1       0.42      0.70      0.53       377

    accuracy                           0.87      3750
   macro avg       0.69      0.80      0.73      3750
weighted avg       0.91      0.87      0.89      3750



In [36]:
LearningQDeep.mainNetwork.save('Trained_model.h5')